# Исследование данных БД сервиса для чтения книг по подписке

**Цель работы:** 

Наша задача —  проанализировать базу данных крупного сервиса для чтения книг по подписке, а в ней — информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных:**

Таблица books

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

**План работы**:
- Обзор данных.
- Решение заданий
- Общий вывод.

## Обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)


In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

In [6]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Решение заданий

### Задание 1

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [11]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT count(book_id)FROM books
           where publication_date >='2000-01-01' '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


In [12]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT count (book_id) FROM books
           where publication_date >='2000-01-01' '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


Всего с 1 января 2000 вышла 821 книга.

### Задание 2

Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [13]:
# чтобы выполнить SQL-запрос, используем Pandas
query = ''' WITH rating_cnt as 
            (SELECT 
                book_id,  
                round(avg(rating),2) as rating_avg 
            FROM ratings 
            GROUP BY book_id),
            
            review_cnt as 
            (SELECT 
                book_id,  
                count(review_id) as review_cnt 
            FROM reviews 
            GROUP BY book_id)

           SELECT 
                a.author, 
                b.title, 
                coalesce(review_cnt, 0) as review_cnt, 
                rating_avg 
           FROM books b
           left join review_cnt as rw on b.book_id=rw.book_id
           left join rating_cnt as ra on b.book_id=ra.book_id
           left join authors as a on b.author_id=a.author_id
           order by review_cnt desc '''
           
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,title,review_cnt,rating_avg
0,Stephenie Meyer,Twilight (Twilight #1),7,3.66
1,Cormac McCarthy,The Road,6,3.77
2,Paulo Coelho/Alan R. Clarke/Özdemir İnce,The Alchemist,6,3.79
3,Mark Haddon,The Curious Incident of the Dog in the Night-Time,6,4.08
4,Lois Lowry,The Giver (The Giver #1),6,3.75
...,...,...,...,...
995,Dr. Seuss/Various,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,Teddy Slater/Ron Dias/Ric González,Disney's Beauty and the Beast (A Little Golden...,0,4.00
997,Edgar Allan Poe/Benjamin F. Fisher,Essential Tales and Poems,0,4.00
998,Faye Perozich/Anne Rice/John Bolton/Daerick Gröss,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Наибольшее количество обзоров получили книги Twilight (Twilight #1) Stephenie Meyer, The Road от Cormac McCarthy и  The Alchemist от Paulo Coelho/Alan R. Clarke/Özdemir İnce.	

### Задание 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT 
                publisher , 
                count(book_id) as book_cnt
            FROM publishers p
            left join books b on p.publisher_id=b.publisher_id 
            WHERE b.num_pages>50 
            GROUP BY publisher
            ORDER BY book_cnt desc 
            limit 1'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,book_cnt
0,Penguin Books,42


Наибольшее число книг толще 50 страниц (исключаем брошюры) выпустило издательство Penguin Books. Всего оно опубликовало 42 книги.

### Задание 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;


In [11]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''WITH rate_cnt as 
            (SELECT 
                book_id,  
                count(rating_id) as rating_cnt 
            FROM ratings 
            GROUP BY book_id
            HAVING count(rating_id)>=50)

            SELECT 
                a.author, 
                round(avg(ra.rating),2) as rating_avg
            FROM authors a

            left join books as b on  b.author_id=a.author_id
            left join ratings as ra on b.book_id=ra.book_id
            join rate_cnt as rt on b.book_id=rt.book_id
            GROUP BY  a.author
            ORDER BY rating_avg desc
            LIMIT 1 '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг 4,29. Учитываются книги с количеством оценок 50 и выше.

### Задание 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


In [12]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''WITH rate_cnt as 
            (SELECT 
                username,  
                count(rating_id) as rating_cnt 
            FROM ratings 
            group by username
            having count(rating_id) >48),
            
            reviews_cnt as 
            (SELECT 
                username,  
                count(review_id) as review_cnt 
            FROM reviews
            group by username)

            SELECT
            avg(review_cnt)
            FROM reviews_cnt rw
            join rate_cnt as ra on ra.username=rw.username
           
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24. 

### Выводы

1. Всего с 1 января 2000 вышла 821 книга.
2. Наибольшее количество обзоров получили книги Twilight (Twilight #1) Stephenie Meyer, The Road от Cormac McCarthy и  The Alchemist от Paulo Coelho/Alan R. Clarke/Özdemir İnce.
3. Наибольшее число книг толще 50 страниц (исключаем брошюры) выпустило издательство Penguin Books. Всего оно опубликовало 42 книги.
4. J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг 4,29. Учитываются книги с количеством оценок 50 и выше.
5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24. 

Рекомендации
Аудитория явно предпочтитает романы, в частности книги  Стефани Мейер, Кормак Маккартни и Паоло Коэлью. Самые высокие оценки у книг Джоан Роулинг. Стоит ориентировать ценностное предложение с учетом интересов этих пользователей.  